In [1]:
# Load the environment variables
import dotenv
dotenv.load_dotenv('.env')

from databricks.labs.doc_qa.llm_utils import PromptTemplate
import pandas as pd
from databricks.labs.doc_qa.evaluators.templated_evaluator import OpenAIEvaluator, AnthropicEvaluator, ParameterDef, NoRetryPolicy, RetryPolicy
from databricks.labs.doc_qa.variables.doc_qa_template_variables import anthropic_grading_template_scale_3, anthropic_grading_template_scale_1
from databricks.labs.doc_qa.variables.doc_qa_template_variables import get_openai_grading_template_and_function


# show debug log for all loggers
import logging
logging.basicConfig(level=logging.INFO)


retry_policy = RetryPolicy(max_retry_on_invalid_result=3, max_retry_on_exception=3)
catch_error = True

openai_grading_prompt, openai_grading_function = get_openai_grading_template_and_function(scale=3, level_of_details=2)
openai_gpt_4_evaluator = OpenAIEvaluator(model="gpt-4", temperature=0.1, 
    grading_prompt_tempate=openai_grading_prompt, 
    input_columns=["question", "answer", "context"], openai_function=openai_grading_function,
    retry_policy=retry_policy)


In [2]:
gpt_35_df = pd.read_csv("pre_grade_datasets/pre_grade_doc_qa_gpt_35.csv")
from databricks.labs.doc_qa.evaluators.templated_evaluator import OpenAIEvaluator

eval_result = openai_gpt_4_evaluator.run_eval(dataset_df=gpt_35_df, concurrency=20, catch_error=catch_error)
result_df = eval_result.to_dataframe()
print(eval_result.summary())

# for each row of result_df, look for the row with same value for question and context from gpt_35_df, and use the "source" column from gpt_35_df as the "source" column for result_df
result_df["source"] = ""
for index, row in result_df.iterrows():
    question = row["question"]
    context = row["context"]
    source = gpt_35_df.loc[(gpt_35_df["question"] == question) & (gpt_35_df["context"] == context)]["source"].values[0]
    result_df.at[index, "source"] = source
# Save to path "post_grade_datasets/doc_qa_gpt_35.csv"
result_df.to_csv("post_grade_datasets/doc_qa_gpt_35.csv", index=False)
result_df

num_rows: 114
num_successful_rows: 114
avg_correctness: 2.991228070175439
avg_comprehensiveness: 2.973684210526316
avg_readability: 2.982456140350877



,is_successful,error_msg,reasoning_for_correctness,correctness,reasoning_for_comprehensiveness,comprehensiveness,reasoning_for_readability,readability,context,question,answer,source
0,True,None,The answer correctly identifies the type used ...,3,The answer is concise and directly answers the...,3,"The answer is clear, concise, and easy to unde...",3,"\n[c](ShortType$.html ""See companion object"")\...",What is the name of the type used in JSON seri...,The name of the type used in JSON serializatio...,https://spark.apache.org/docs/latest/api/scala...
1,True,None,The answer correctly explains what a DenseVect...,3,"The answer is comprehensive, covering the defi...",3,The answer is well-structured and easy to read.,3,"\n[c](DenseVector$.html ""See companion object""...",What is DenseVector?,A DenseVector is a type of vector in Apache Sp...,https://spark.apache.org/docs/latest/api/scala...
2,True,None,The answer correctly identifies the return val...,3,The answer is concise and directly answers the...,3,"The answer is clear, concise, and easy to unde...",3,\n![]()\n cube\n======\n`cube.Rd` \n Create a ...,What is the return value of `cube`?,The return value of `cube` is a GroupedData ob...,https://spark.apache.org/docs/latest/api/R/ref...
3,True,None,The answer provides a correct example of using...,2,The answer is not comprehensive as it does not...,1,The answer is readable and well-formatted as J...,2,"\n FPGrowth\n [¶](#fpgrowth ""Permalink to this...",8. Can you provide an example of using FPGrowt...,Sure! Here is an example of how to use FPGrowt...,https://spark.apache.org/docs/latest/api/pytho...
4,True,None,The answer correctly identifies the value memb...,3,The answer provides a comprehensive list of th...,3,The answer is well-structured and easy to read.,3,\n c\n \n[org](../../../../../index.html) \n ....,What are the value members of RuntimeInfo?,The value members of the RuntimeInfo class are...,https://spark.apache.org/docs/latest/api/scala...
...,...,...,...,...,...,...,...,...,...,...,...,...
109,True,None,The answer correctly lists all the linear supe...,3,The answer is comprehensive as it covers all t...,3,The answer is well-structured and easy to read.,3,"\n[c](GradientDescent$.html ""See companion obj...",7. What are the linear supertypes of Gradient ...,The linear supertypes of Gradient Descent in S...,https://spark.apache.org/docs/latest/api/scala...
110,True,None,The answer correctly explains the BisectingKMe...,3,The answer covers all the main aspects of the ...,2,The answer is well-structured and easy to unde...,2,\n p\n \n[org](../../../../index.html) \n .\n ...,1. What is BisectingKMeans algorithm in Spark?,The BisectingKMeans algorithm in Spark is a cl...,https://spark.apache.org/docs/latest/api/scala...
111,True,None,The answer correctly explains how to use the o...,3,The answer is comprehensive. It explains the s...,3,The answer is well-structured and easy to read...,3,\n c\n \n[org](../../../index.html) \n .\n [ap...,9. How can we add an output option for the und...,To add an output option for the underlying dat...,https://spark.apache.org/docs/latest/api/scala...
112,True,None,The answer correctly explains how to fit a mod...,3,"The answer is comprehensive, covering all aspe...",3,The answer is well-structured and easy to read...,3,"\n[c](OneHotEncoder$.html ""See companion objec...",10. How can you fit a model using OneHotEncode...,"To fit a model using OneHotEncoder in Spark, y...",https://spark.apache.org/docs/latest/api/scala...


In [3]:
vicuna_df = pd.read_csv("pre_grade_datasets/pre_grade_doc_qa_vicuna_33b.csv")
from databricks.labs.doc_qa.evaluators.templated_evaluator import OpenAIEvaluator

eval_result = openai_gpt_4_evaluator.run_eval(dataset_df=vicuna_df, concurrency=20, catch_error=catch_error)
result_df = eval_result.to_dataframe()
print(eval_result.summary())

# for each row of result_df, look for the row with same value for question and context from gpt_35_df, and use the "source" column from gpt_35_df as the "source" column for result_df
result_df["source"] = ""
for index, row in result_df.iterrows():
    question = row["question"]
    context = row["context"]
    source = vicuna_df.loc[(vicuna_df["question"] == question) & (vicuna_df["context"] == context)]["source"].values[0]
    result_df.at[index, "source"] = source
result_df.to_csv("post_grade_datasets/doc_qa_vicuna_33b.csv", index=False)
result_df

num_rows: 115
num_successful_rows: 115
avg_correctness: 1.434782608695652
avg_comprehensiveness: 1.434782608695652
avg_readability: 1.5130434782608695



,is_successful,error_msg,reasoning_for_correctness,correctness,reasoning_for_comprehensiveness,comprehensiveness,reasoning_for_readability,readability,context,question,answer,source
0,True,None,The answer correctly states that the function ...,3,"The answer covers all aspects of the question,...",3,The answer is well-structured and easy to unde...,3,\n pyspark.pandas.Series.str.cat\n [¶](#pyspar...,How does pyspark.pandas.Series.str.cat work?,Answer: `pyspark.pandas.Series.str.cat` is an...,https://spark.apache.org/docs/latest/api/pytho...
1,True,None,The answer correctly explains what named aggre...,3,The answer provides a comprehensive explanatio...,3,The answer is well-structured and easy to unde...,3,\n pyspark.pandas.groupby.DataFrameGroupBy.agg...,What is named aggregation in PySpark?,Named aggregation in PySpark refers to the pr...,https://spark.apache.org/docs/latest/api/pytho...
2,True,None,The answer correctly identifies the return val...,3,The answer is concise and directly answers the...,3,The answer is clear and easy to understand.,3,\n![]()\n cube\n======\n`cube.Rd` \n Create a ...,What is the return value of `cube`?,The return value of the `cube` function is a ...,https://spark.apache.org/docs/latest/api/R/ref...
3,True,None,The answer is completely incorrect and doesn't...,0,The answer doesn't provide any information rel...,0,The answer is readable but it's not relevant t...,1,"\n[c](ShortType$.html ""See companion object"")\...",What is the name of the type used in JSON seri...,NaN,https://spark.apache.org/docs/latest/api/scala...
4,True,None,The answer is completely irrelevant to the que...,0,The answer does not provide any information re...,0,The answer is not readable and does not make a...,0,"\n FPGrowth\n [¶](#fpgrowth ""Permalink to this...",8. Can you provide an example of using FPGrowt...,I ...,https://spark.apache.org/docs/latest/api/pytho...
...,...,...,...,...,...,...,...,...,...,...,...,...
110,True,None,The answer is completely irrelevant to the que...,0,The answer does not provide any information re...,0,The answer is not readable at all. It is just ...,0,"\n[o](SparkContext.html ""See companion class"")...",7. How can the getOrCreate() function be used ...,A I The The The The The The ...,https://spark.apache.org/docs/latest/api/scala...
111,True,None,The answer is completely irrelevant to the que...,0,The answer does not provide any information re...,0,The answer is not readable and does not make a...,0,\n c\n \n[org](../../../index.html) \n .\n [ap...,9. What are the linear supertypes of SequenceF...,A I ...,https://spark.apache.org/docs/latest/api/scala...
112,True,None,The answer is completely incorrect and irrelev...,0,The answer does not provide any information re...,0,The answer is not readable and does not make a...,0,\n QuantileDiscretizer\n [¶](#quantilediscreti...,7. What is the precision of the approximation ...,A I ...,https://spark.apache.org/docs/latest/api/pytho...
113,True,None,The answer is completely incorrect and doesn't...,0,The answer is not comprehensive at all. It doe...,0,The answer is completely unreadable. It's full...,0,\n Source code for pyspark.mllib.random\n=====...,6. How can you generate an RDD of vectors cont...,A I ...,https://spark.apache.org/docs/latest/api/pytho...
